In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from tqdm import tqdm

# 1. Verileri yükle
train = pd.read_csv("train.csv")
test = pd.read_csv("testFeatures.csv")

# 2. Tarihleri dönüştür
train["tarih"] = pd.to_datetime(train["tarih"])
test["tarih"] = pd.to_datetime(test["tarih"])

# 3. Prophet modellerini saklayacağımız sözlük
prophet_models = {}

# 4. Eğitim segmentlerini belirle
segments = train.groupby(["ürün", "ürün üretim yeri", "market", "şehir", ])

# 5. Her segment için Prophet modelini eğit (log1p dönüşümü ile)
print("Modeller eğitiliyor...")
for segment_key, segment_df in tqdm(segments):
    if len(segment_df) < 10:
        continue  # Veri azsa geç

    prophet_df = segment_df.rename(columns={"tarih": "ds", "ürün fiyatı": "y"})[["ds", "y"]]
    
    # LOG1P dönüşümü
    prophet_df["y"] = np.log1p(prophet_df["y"])
    
    try:
        model = Prophet()
        model.fit(prophet_df)
        prophet_models[segment_key] = model
    except:
        continue

# 6. Test verileri için tahmin yap
predictions = []

print("Tahminler yapılıyor...")
for idx, row in tqdm(test.iterrows(), total=len(test)):
    segment_key = (row["ürün"], row["ürün üretim yeri"], row["market"], row["şehir"])
    tarih = row["tarih"]

    model = prophet_models.get(segment_key)
    
    if model:
        future_df = pd.DataFrame({"ds": [tarih]})
        try:
            forecast = model.predict(future_df)
            
            # EXPm1 ile ters dönüşüm
            predicted_price = np.expm1(forecast["yhat"].values[0])
            predictions.append(predicted_price)
        except:
            predictions.append(None)
    else:
        predictions.append(None)  # Model bulunamazsa boş bırak

# Tahminleri test veri setine ekle
test = test.reset_index(drop=True)
test["ürün fiyatı"] = predictions

# id'yi test sırasına göre sırala
test["id"] = test.index

# Sadece gerekli sütunları al
submission = test[["id", "ürün fiyatı"]]

# CSV olarak kaydet
submission.to_csv("submission_prophet_v5.csv", index=False)


Modeller eğitiliyor...


  0%|                                                  | 0/3792 [00:00<?, ?it/s]14:54:06 - cmdstanpy - INFO - Chain [1] start processing
14:54:06 - cmdstanpy - INFO - Chain [1] done processing
  0%|                                          | 1/3792 [00:00<20:25,  3.09it/s]14:54:06 - cmdstanpy - INFO - Chain [1] start processing
14:54:06 - cmdstanpy - INFO - Chain [1] done processing
  0%|                                          | 2/3792 [00:00<13:50,  4.56it/s]14:54:06 - cmdstanpy - INFO - Chain [1] start processing
14:54:06 - cmdstanpy - INFO - Chain [1] done processing
  0%|                                          | 3/3792 [00:00<12:36,  5.01it/s]14:54:06 - cmdstanpy - INFO - Chain [1] start processing
14:54:06 - cmdstanpy - INFO - Chain [1] done processing
  0%|                                          | 4/3792 [00:00<12:07,  5.21it/s]14:54:06 - cmdstanpy - INFO - Chain [1] start processing
14:54:07 - cmdstanpy - INFO - Chain [1] done processing
  0%|                              